In [13]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder

# OpenAI 챗봇 모델 설정
llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question}, 
        {"output": result.content}
    )
    print(result)


In [14]:
invoke_chain("My name is Jacob")

content='Hello Jacob! How can I assist you today?'


In [15]:
invoke_chain("Can you guess my name ?")

content='Based on the information you provided, your name is Jacob. Is that correct?'


In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder

# OpenAI 챗봇 모델 설정
llm = ChatOpenAI(temperature=0.1)

# 대화 요약 버퍼 메모리 초기화
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,  # 저장할 최대 토큰 수 제한
    return_messages=True,  # 메시지 반환 활성화
)

# 프롬프트 템플릿 설정
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="history"),  # 메시지 플레이스홀더를 통한 동적 대화 내용 삽입
        ("human", "{question}"),
    ]
)

# 메모리에서 대화 내용 로드하는 함수
def load_memory(_):
    return memory.load_memory_variables({})["history"]

# RunnablePassthrough를 사용하여 체인 생성
chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

# 체인 실행 함수
def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question}, 
        {"output": result.content}
    )
    print(result)

# 체인을 사용하여 질문 실행
invoke_chain("My name is Jacob")
invoke_chain("Can you guess my name ?")

# 이 스크립트는 Langchain의 다양한 컴포넌트를 활용하여 
# 동적 메모리 관리와 맥락을 고려한 대화가 가능한 AI 시스템을 구축합니다. 
# 복잡한 대화 관리와 메모리 최적화를 위한 고급 기법을 적용하여, 
# 사용자와의 상호작용을 통해 AI가 지속적으로 발전할 수 있는 구조를 제공합니다.


# User Query -> invoke_chain()
#     |
#     v
# RunnablePassthrough - Loads conversation history
#     |                   from ConversationSummaryBufferMemory
#     v
# ChatPromptTemplate - Generates dynamic prompt including
#     |                conversation history and the current question
#     v
# ChatOpenAI (LLM) - Processes the prompt and generates a response
#     |
#     v
# ConversationSummaryBufferMemory - Saves the current input/output
#     |                              pair to the conversation history
#     v
# Return Response - The generated response is returned to the user


